# Freeswitch Audio Recordings Traffic Analysis 

## Altanai Bisht 
( abisht@seattleu.import os 
os.environ['JAVA_HOME']="C:\\Users\\abisht\\.jdks\\corretto-11.0.15"
print(os.environ['JAVA_HOME'])m)

License GPL 3

In [ ]:
import os 
os.environ['JAVA_HOME']="C:\\Users\\abisht\\.jdks\\corretto-11.0.15"
print(os.environ['JAVA_HOME'])

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics 
spark = SparkSession.builder.getOrCreate()

# Call Recorded Audio File 

- wav

In [31]:
import scipy
import os.path, time, datetime
import platform


In [29]:
audiopath= 'C:\\Users\\abisht\\VoIPCallLogs_BigDataSet\\audiofiles\\*.wav'

In [30]:
# creation_date(audiopath)
# print("Created: %s" % creation_date(audiopath))
rtime= os.stat(audiopath).st_mtime
print(rtime)
print("Date created:",datetime.datetime.fromtimestamp(rtime))

1654011853.8700197
Date created: 2022-05-31 08:44:13.870020


In [ ]:
from pyspark.sql.functions import regexp_extract

logs_df = base_df.select(regexp_extract('value', ts_pattern, 1).alias('timestamp'))
logs_df.show(10, truncate=True)
print((logs_df.count(), len(logs_df.columns)))